In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 加载数据
df = pd.DataFrame(pd.read_csv("../Data/watermelon3.0.csv", encoding="ansi"))
df.drop(labels=["编号"], axis=1, inplace=True)  # 删除编号这一列，inplace=True表示直接在原对象修改
features = ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '密度', '含糖量']
# 提取特征和标签
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
# 转码
X = pd.DataFrame(X, columns=features)
# 独热编码
X_encoded = pd.get_dummies(X,columns = ["色泽","根蒂","敲声","纹理","脐部","触感"])
new_features = list(X_encoded.columns)
X_encoded = np.array(X_encoded, dtype=np.float64)
# 标签数值化
for i in range(len(y)):
    if y[i] == '是':
        y[i] = 1
    else:
        y[i] = 0
y_encoded = np.array(y, dtype=np.int64).reshape(-1,1)
# 拼接特征和标签
dataset = np.concatenate((X_encoded, y_encoded), axis=1)
print(dataset)   
     




[[0.697 0.46  0.    0.    1.    0.    0.    1.    0.    1.    0.    0.
  1.    0.    1.    0.    0.    1.    0.    1.   ]
 [0.774 0.376 1.    0.    0.    0.    0.    1.    1.    0.    0.    0.
  1.    0.    1.    0.    0.    1.    0.    1.   ]
 [0.634 0.264 1.    0.    0.    0.    0.    1.    0.    1.    0.    0.
  1.    0.    1.    0.    0.    1.    0.    1.   ]
 [0.608 0.318 0.    0.    1.    0.    0.    1.    1.    0.    0.    0.
  1.    0.    1.    0.    0.    1.    0.    1.   ]
 [0.556 0.215 0.    1.    0.    0.    0.    1.    0.    1.    0.    0.
  1.    0.    1.    0.    0.    1.    0.    1.   ]
 [0.403 0.237 0.    0.    1.    0.    1.    0.    0.    1.    0.    0.
  1.    0.    0.    0.    1.    0.    1.    1.   ]
 [0.481 0.149 1.    0.    0.    0.    1.    0.    0.    1.    0.    0.
  0.    1.    0.    0.    1.    0.    1.    1.   ]
 [0.437 0.211 1.    0.    0.    0.    1.    0.    0.    1.    0.    0.
  1.    0.    0.    0.    1.    1.    0.    1.   ]
 [0.666 0.091 1.    0.  